In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

# Functions for Demographics
def calculate_age(dob, intime):
    """Calculate age at time of ICU stay given date of birth and ICU intime."""
    return intime.year - dob.year - ((intime.month, intime.day) < (dob.month, dob.day))

def categorize_age(age):
    """Categorize age into one of four bins."""
    if 15 <= age <= 29:
        return '15-29'
    elif 30 <= age <= 49:
        return '30-49'
    elif 50 <= age <= 69:
        return '50-69'
    else:
        return '70-89'

def categorize_ethnicity(ethnicity):
    """Simplify ethnicity descriptions."""
    ethnicity = ethnicity.upper()
    if ethnicity in ['WHITE', 'WHITE - RUSSIAN', 'WHITE - OTHER EUROPEAN', 'WHITE - BRAZILIAN', 'WHITE - EASTERN EUROPEAN']:
        return 'White'
    elif ethnicity in ['BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/HAITIAN', 'BLACK/AFRICAN', 'CARIBBEAN ISLAND']:
        return 'Black'
    elif ethnicity in ['HISPANIC OR LATINO', 'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - DOMINICAN', 'HISPANIC/LATINO - MEXICAN']:
        return 'Hispanic'
    elif ethnicity in ['ASIAN', 'ASIAN - CHINESE', 'ASIAN - INDIAN']:
        return 'Asian'
    else:
        return 'Other'

def categorize_insurance(insurance):
    """Categorize insurance based on keyword matching."""
    ins = insurance.upper()
    if 'MEDICARE' in ins:
        return 'Medicare'
    elif 'PRIVATE' in ins:
        return 'Private'
    elif 'MEDICAID' in ins:
        return 'Medicaid'
    elif 'SELF PAY' in ins:
        return 'Self Pay'
    else:
        return 'Government'

# Functions for Text Preprocessing
def preprocess1(x):
    """
    Remove extra characters, numeric bullet points, and standardize abbreviations.
    """
    y = re.sub(r'\[(.*?)\]', '', x)         
    y = re.sub(r'[0-9]+\.', '', y)           
    y = re.sub(r'dr\.', 'doctor', y)
    y = re.sub(r'm\.d\.', 'md', y)
    y = re.sub(r'admission date:', '', y)
    y = re.sub(r'discharge date:', '', y)
    y = re.sub(r'--|__|==', '', y)
    return y

def preprocessing(df):
    """
    Preprocess the 'TEXT' column of a dataframe: remove newlines, extra whitespace, lower case, and apply cleanup.
    """
    df = df.copy()
    df['TEXT'] = df['TEXT'].fillna(' ')
    df['TEXT'] = df['TEXT'].str.replace('\n', ' ', regex=False)
    df['TEXT'] = df['TEXT'].str.replace('\r', ' ', regex=False)
    df['TEXT'] = df['TEXT'].apply(str.strip)
    df['TEXT'] = df['TEXT'].str.lower()
    df['TEXT'] = df['TEXT'].apply(lambda x: preprocess1(x))
    return df

# Functions for Outcome Calculations
def calculate_short_term_mortality(icu_stays):
    """
    Create a binary column 'short_term_mortality' based on whether DEATHTIME is present.
    """
    icu_stays['short_term_mortality'] = icu_stays['DEATHTIME'].notnull().astype(int)
    return icu_stays

def calculate_readmission(icu_stays):
    """
    Create a binary column 'readmission_within_30_days' based on whether the next ICU admission is within 30 days.
    """
    required = ['DISCHTIME', 'INTIME', 'hadm_id']
    for col in required:
        if col not in icu_stays.columns:
            raise KeyError(f"Column {col} is missing in the input data.")
    
    # Sort by subject, admission time, and ICU intime
    icu_stays = icu_stays.sort_values(by=['subject_id', 'ADMITTIME', 'INTIME'])
    
    # Get the discharge time of the current admission
    icu_stays['current_admission_dischtime'] = icu_stays.groupby(['subject_id', 'hadm_id'])['DISCHTIME'].transform('first')
    
    # For each subject, get the next ICU admission's intime and corresponding hadm_id
    icu_stays['next_admission_icu_intime'] = icu_stays.groupby('subject_id')['INTIME'].shift(-1)
    icu_stays['next_hadm_id'] = icu_stays.groupby('subject_id')['hadm_id'].shift(-1)
    
    # Calculate readmission if the next ICU admission happens within 30 days of the current discharge time
    icu_stays['readmission_within_30_days'] = (
        (icu_stays['next_admission_icu_intime'] - icu_stays['current_admission_dischtime']).dt.days <= 30
    ).astype(int)
    icu_stays['readmission_within_30_days'] = icu_stays['readmission_within_30_days'].fillna(0).astype(int)
    return icu_stays

# Functions for Splitting Text into Chunks
def split_text_to_chunks(text, chunk_size=512):
    """
    Split a text into chunks of a given token size.
    Tokens are defined by whitespace.
    """
    tokens = text.split()
    chunks = [' '.join(tokens[i:i + chunk_size]) for i in range(0, len(tokens), chunk_size)]
    return chunks

def split_into_512_token_columns(text, chunk_size=512):
    """
    Given a text, return a Series with one column per chunk.
    """
    chunks = split_text_to_chunks(text, chunk_size)
    chunk_dict = {}
    for i, chunk in enumerate(chunks):
        chunk_dict[f"note_chunk_{i+1}"] = chunk
    return pd.Series(chunk_dict)

# Data Loading
admissions_path = 'ADMISSIONS.csv.gz'
icustays_path   = 'ICUSTAYS.csv.gz'
patients_path   = 'PATIENTS.csv.gz'
notes_path      = 'NOTEEVENTS.csv.gz'

# Read Admissions data
df_adm = pd.read_csv(admissions_path, compression='gzip', low_memory=False, 
                     usecols=['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'INSURANCE'])
# Read ICU stays data
df_icustays = pd.read_csv(icustays_path, compression='gzip', low_memory=False, 
                          usecols=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'INTIME', 'OUTTIME'])
# Read Patients data
df_patients = pd.read_csv(patients_path, compression='gzip', low_memory=False, 
                          usecols=['SUBJECT_ID', 'DOB', 'GENDER'])
# Read Notes data
df_notes = pd.read_csv(notes_path, compression='gzip', low_memory=False, 
                       usecols=['SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'TEXT'])

# Admissions datetime conversion
df_adm['ADMITTIME'] = pd.to_datetime(df_adm['ADMITTIME'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_adm['DISCHTIME'] = pd.to_datetime(df_adm['DISCHTIME'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_adm['DEATHTIME'] = pd.to_datetime(df_adm['DEATHTIME'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# ICU stays datetime conversion
df_icustays['INTIME'] = pd.to_datetime(df_icustays['INTIME'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_icustays['OUTTIME'] = pd.to_datetime(df_icustays['OUTTIME'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Patients DOB conversion
df_patients['DOB'] = pd.to_datetime(df_patients['DOB'], format='%Y-%m-%d', errors='coerce')

# Notes datetime conversion
df_notes['CHARTDATE'] = pd.to_datetime(df_notes['CHARTDATE'], format='%Y-%m-%d', errors='coerce')

# Rename columns for consistency
df_adm.rename(columns={'SUBJECT_ID': 'subject_id', 'HADM_ID': 'hadm_id'}, inplace=True)
df_icustays.rename(columns={'SUBJECT_ID': 'subject_id', 'HADM_ID': 'hadm_id'}, inplace=True)
df_patients.rename(columns={'SUBJECT_ID': 'subject_id'}, inplace=True)
df_notes.rename(columns={'SUBJECT_ID': 'subject_id', 'HADM_ID': 'hadm_id'}, inplace=True)

# Merge admissions with ICU stays
df_icu = pd.merge(df_adm, df_icustays, on=['subject_id', 'hadm_id'], how='inner')

# Merge patient demographics (DOB, GENDER) into the ICU dataframe
df_icu = pd.merge(df_icu, df_patients[['subject_id', 'DOB', 'GENDER']], on='subject_id', how='left')

# Calculate patient age at the time of ICU INTIME
df_icu['age'] = df_icu.apply(lambda row: calculate_age(row['DOB'], row['INTIME']) 
                             if pd.notnull(row['DOB']) and pd.notnull(row['INTIME']) else np.nan, axis=1)
df_icu['age_category'] = df_icu['age'].apply(lambda x: categorize_age(x) if pd.notnull(x) else 'Unknown')

# Categorize ethnicity and insurance
df_icu['ethnicity_category'] = df_icu['ETHNICITY'].apply(lambda x: categorize_ethnicity(x) if pd.notnull(x) else 'Other')
df_icu['insurance_category'] = df_icu['INSURANCE'].apply(lambda x: categorize_insurance(x) if pd.notnull(x) else 'Other')

# Ensure gender is in a consistent format: male or female
df_icu['gender'] = df_icu['GENDER'].str.lower().apply(lambda x: 'male' if 'm' in x else ('female' if 'f' in x else x))

# Short-term mortality (if DEATHTIME exists, label as 1)
df_icu = calculate_short_term_mortality(df_icu)

# Readmission within 30 days
df_icu = calculate_readmission(df_icu)

# Select First ICU Stay per Patient
df_first_icu = df_icu.sort_values(by='INTIME').groupby('subject_id').first().reset_index()


# Select notes corresponding to first ICU admissions based on hadm_id
first_icu_notes = df_notes[df_notes['hadm_id'].isin(df_first_icu['hadm_id'])]

# Aggregate notes by subject and admission
notes_agg = first_icu_notes.groupby(['subject_id', 'hadm_id'])['TEXT'].apply(lambda texts: " ".join(texts)).reset_index()

# Clean the text using preprocessing
notes_agg = preprocessing(notes_agg)

# Merge aggregated notes into the first ICU dataframe
df_first_icu = pd.merge(df_first_icu, notes_agg, on=['subject_id', 'hadm_id'], how='left')
df_first_icu['TEXT'] = df_first_icu['TEXT'].fillna('')

# Check the maximum size (number of characters and word count) of the concatenated notes
df_first_icu['note_length_chars'] = df_first_icu['TEXT'].apply(len)
df_first_icu['note_length_words'] = df_first_icu['TEXT'].apply(lambda x: len(x.split()))
max_chars = df_first_icu['note_length_chars'].max()
max_words = df_first_icu['note_length_words'].max()
print(f"Maximum note length (chars): {max_chars}")
print(f"Maximum note length (words): {max_words}")

# Overall shape of the final structured dataset
print("Final dataset shape:", df_first_icu.shape)

# Count positive outcomes for short-term mortality and readmission
num_mortality_positive = df_first_icu['short_term_mortality'].sum()
num_readmission_positive = df_first_icu['readmission_within_30_days'].sum()
print("Number of patients with short-term mortality (positive):", num_mortality_positive)
print("Number of patients with readmission within 30 days (positive):", num_readmission_positive)

# Save the structured dataset (first ICU stays with demographics and outcomes)
df_first_icu.to_csv('final_first_icu_dataset.csv', index=False)

# Check for duplicate patient entries
duplicate_patients = df_first_icu[df_first_icu.duplicated(subset=['subject_id'], keep=False)]
print("Number of duplicate patient entries:", duplicate_patients.shape[0])
if duplicate_patients.shape[0] > 0:
    print("Duplicate patient entries found:")
    print(duplicate_patients[['subject_id']].drop_duplicates())
else:
    print("No duplicate patient entries found.")

# Apply the splitting function to each row in the 'TEXT' column.
df_note_chunks = df_first_icu['TEXT'].apply(split_into_512_token_columns)

# Concatenate the note chunk columns with the main dataframe.
df_final = pd.concat([df_first_icu, df_note_chunks], axis=1)

# Save the final unstructured dataset as CSV.
df_final.to_csv('final_unstructured.csv', index=False)
print("Final unstructured dataset shape:", df_final.shape)

# Read the Unstructured Dataset
unstructured_file = 'final_unstructured.csv'
unstructured_df = pd.read_csv(unstructured_file, engine='python', on_bad_lines='skip')
print(f"Unstructured data shape: {unstructured_df.shape}")

# Read the Structured Dataset (assumed to be saved as structured_first_icu_stays.csv)
structured_file = 'structured_first_icu_stays.csv'
structured_df = pd.read_csv(structured_file)
print(f"Structured data shape: {structured_df.shape}")

# Identify the common subject IDs between unstructured and structured datasets.
unstructured_ids = set(unstructured_df['subject_id'].unique())
structured_ids   = set(structured_df['subject_id'].unique())
common_ids = unstructured_ids.intersection(structured_ids)
print(f"Number of common subject IDs: {len(common_ids)}")

# Filter unstructured dataset to include only common subject IDs.
filtered_unstructured_df = unstructured_df[unstructured_df['subject_id'].isin(common_ids)].copy()
print(f"Filtered unstructured data shape: {filtered_unstructured_df.shape}")

# Save the filtered unstructured dataset.
filtered_unstructured_df.to_csv('filtered_unstructured.csv', index=False)
print("Filtered unstructured dataset saved as 'filtered_unstructured.csv'.")


Maximum note length (chars): 3859128
Maximum note length (words): 525957
Final dataset shape: (46476, 25)
Number of patients with short-term mortality (positive): 4460
Number of patients with readmission within 30 days (positive): 4055
Number of duplicate patient entries: 0
No duplicate patient entries found.
Final unstructured dataset shape: (46476, 1053)
Unstructured data shape: (43124, 1053)
Structured data shape: (36615, 39)
Number of common subject IDs: 33942
Filtered unstructured data shape: (33942, 1053)
Filtered unstructured dataset saved as 'filtered_unstructured.csv'.


In [12]:
# Count positive outcomes for short-term mortality and readmission
num_mortality_positive = filtered_unstructured_df['short_term_mortality'].sum()
num_readmission_positive = filtered_unstructured_df['readmission_within_30_days'].sum()
print("Number of patients with short-term mortality (positive):", num_mortality_positive)
print("Number of patients with readmission within 30 days (positive):", num_readmission_positive)


Number of patients with short-term mortality (positive): 3525
Number of patients with readmission within 30 days (positive): 3033
